## 1. Data Import

In [1]:
import csv
import pandas as pd
import geopandas as gpd
import fiona
from os import listdir
import re

df = gpd.read_file('ej-0424/ej.shp')
#df = df.set_index('ID').fillna(0)
display(df.head())

,ID,ACSTOTPOP,ACSIPOVBAS,ACSEDUCBAS,ACSTOTHH,ACSTOTHU,MINORPOP,MINORPCT,LOWINCOME,LOWINCPCT,...,T_PM25_B6,T_PM25_P2,T_PM25_P6,AREALAND,AREAWATER,NPL_CNT,TSDF_CNT,Shape_Leng,Shape_Area,geometry
0,170310101001,639,639,515,312,376,443,0.693271,247,0.386541,...,29%ile,87%ile,79%ile,62463.0,0.0,0,0,1689.212502,113261.122684,POLYGON ((-9759460.328199999 5164183.183399998...
1,170310101002,1768,1751,1116,833,955,1202,0.679864,1217,0.695031,...,86%ile,95%ile,92%ile,183315.0,0.0,0,0,2659.049626,332392.122294,POLYGON ((-9760181.790100001 5164417.844300002...
2,170310101003,1981,1881,1529,1061,1292,994,0.501767,620,0.329612,...,69%ile,77%ile,67%ile,133973.0,0.0,0,0,3951.080271,242919.062756,"POLYGON ((-9759674.952299999 5164425.9362, -97..."
3,170310102011,1417,1417,1025,619,742,916,0.646436,704,0.496824,...,75%ile,89%ile,88%ile,95922.0,0.0,0,0,1786.028121,173888.451215,POLYGON ((-9760948.113600001 5163250.440399997...
4,170310102012,4641,4628,3064,1673,1810,3615,0.778927,2569,0.555099,...,98%ile,94%ile,93%ile,259326.0,0.0,0,0,3623.548693,470159.705671,"POLYGON ((-9761011.5658 5163899.829499997, -97..."


In [2]:
df.head()

,ID,ACSTOTPOP,ACSIPOVBAS,ACSEDUCBAS,ACSTOTHH,ACSTOTHU,MINORPOP,MINORPCT,LOWINCOME,LOWINCPCT,...,T_PM25_B6,T_PM25_P2,T_PM25_P6,AREALAND,AREAWATER,NPL_CNT,TSDF_CNT,Shape_Leng,Shape_Area,geometry
0,170310101001,639,639,515,312,376,443,0.693271,247,0.386541,...,29%ile,87%ile,79%ile,62463.0,0.0,0,0,1689.212502,113261.122684,POLYGON ((-9759460.328199999 5164183.183399998...
1,170310101002,1768,1751,1116,833,955,1202,0.679864,1217,0.695031,...,86%ile,95%ile,92%ile,183315.0,0.0,0,0,2659.049626,332392.122294,POLYGON ((-9760181.790100001 5164417.844300002...
2,170310101003,1981,1881,1529,1061,1292,994,0.501767,620,0.329612,...,69%ile,77%ile,67%ile,133973.0,0.0,0,0,3951.080271,242919.062756,"POLYGON ((-9759674.952299999 5164425.9362, -97..."
3,170310102011,1417,1417,1025,619,742,916,0.646436,704,0.496824,...,75%ile,89%ile,88%ile,95922.0,0.0,0,0,1786.028121,173888.451215,POLYGON ((-9760948.113600001 5163250.440399997...
4,170310102012,4641,4628,3064,1673,1810,3615,0.778927,2569,0.555099,...,98%ile,94%ile,93%ile,259326.0,0.0,0,0,3623.548693,470159.705671,"POLYGON ((-9761011.5658 5163899.829499997, -97..."


In [3]:
df.shape

(3992, 368)

In [4]:
# get field descriptions
df_dict = pd.read_excel('EJScreen_Index_DescriptionsV4_Pub.xlsx')

### 1*. remove the blockgroups in chicago

In [5]:
df_chicago = pd.read_csv('target_blockgroups.csv')

In [6]:
df_chicago.shape

(2179, 2)

In [7]:
df_chicago.head()

,id,Unnamed: 1
0,170310101001,NaN
1,170310101002,NaN
2,170310101003,NaN
3,170310102011,NaN
4,170310102012,NaN


In [8]:
df_chicago = df_chicago['id'].astype(str)

In [9]:
df_chicago.head()

0    170310101001
1    170310101002
2    170310101003
3    170310102011
4    170310102012
Name: id, dtype: object

In [10]:
df_chicago = df_chicago.tolist()

In [11]:
df = df.loc[~df["ID"].isin(df_chicago)]

In [12]:
df.shape

(1813, 368)

In [13]:
df = df.set_index('ID').fillna(0)

### 2. Process environmental columns

In [14]:
env_indicators = [
    'DSLPM',
    'CANCER',
    'RESP',
    'PTRAF',
    'PWDIS',
    'PNPL',
    'PRMP',
    'PTSDF',
    'OZONE',
    'PM25',
    'PRE1960PCT']

env_dict = {
    'CANCER': 'CANCR',
    'PRE1960PCT': 'LDPNT'
}

#display(df_dict[df_dict.FIELD_NAME.isin(env_indicators)])

#### 2a. Calculate percentile

In [15]:
from scipy import stats

c_env_df = df.filter(env_indicators).apply(lambda series: series.apply(lambda x: stats.percentileofscore(series, x, kind='strict')))
c_env_df.rename(lambda x: 'C_' + env_dict.get(x,x), axis='columns', inplace=True)

display(c_env_df.head())

,C_DSLPM,C_CANCR,C_RESP,C_PTRAF,C_PWDIS,C_PNPL,C_PRMP,C_PTSDF,C_OZONE,C_PM25,C_LDPNT
ID,,,,,,,,,,,
170317702011,73.138445,37.948152,60.121346,90.623276,31.825703,51.958081,60.066189,90.678434,19.966906,35.466078,6.067292
170317702012,73.138445,37.948152,60.121346,20.849421,32.542747,46.221732,64.313293,94.760066,19.966906,35.466078,12.961942
170317702013,73.138445,37.948152,60.121346,97.738555,33.370105,53.557639,43.849972,86.486486,19.966906,35.466078,10.204082
170317702021,24.489796,38.113624,57.970215,4.412576,34.859349,45.173745,70.711528,96.359625,24.986211,29.564258,13.623828
170317702022,24.489796,38.113624,57.970215,22.669608,32.984004,47.104247,61.776062,92.443464,24.986211,29.564258,12.024269


#### 2b. bin percentiles into quintiles

In [16]:
bins = [-1,20,40,60,80,100]
labels = [1,2,3,4,5]

s_env_df = c_env_df.apply(lambda series: pd.cut(series, bins, labels=labels).values.add_categories(0).astype(int))
s_env_df.rename(lambda x: x.replace('C_', 'S_'), axis='columns', inplace=True)

display(s_env_df.head())

,S_DSLPM,S_CANCR,S_RESP,S_PTRAF,S_PWDIS,S_PNPL,S_PRMP,S_PTSDF,S_OZONE,S_PM25,S_LDPNT
ID,,,,,,,,,,,
170317702011,4,2,4,5,2,3,4,5,1,2,1
170317702012,4,2,4,2,2,3,4,5,1,2,1
170317702013,4,2,4,5,2,3,3,5,1,2,1
170317702021,2,2,3,1,2,3,4,5,2,2,1
170317702022,2,2,3,2,2,3,4,5,2,2,1


#### 2c. sum up the qunitiles and bin them

In [17]:
s_env_df['S_ENV_TOTAL'] = s_env_df.sum(axis=1)

series = s_env_df['S_ENV_TOTAL']
s_env_df['C_ENV_TOTAL'] = series.apply(lambda x: stats.percentileofscore(series, x, kind='strict'))
s_env_df['S_ENV_OVERALL'] = pd.cut(s_env_df['C_ENV_TOTAL'], bins, labels=labels).astype(int)

display(s_env_df.head())

,S_DSLPM,S_CANCR,S_RESP,S_PTRAF,S_PWDIS,S_PNPL,S_PRMP,S_PTSDF,S_OZONE,S_PM25,S_LDPNT,S_ENV_TOTAL,C_ENV_TOTAL,S_ENV_OVERALL
ID,,,,,,,,,,,,,,
170317702011,4,2,4,5,2,3,4,5,1,2,1,33,42.360728,3
170317702012,4,2,4,2,2,3,4,5,1,2,1,30,29.784887,2
170317702013,4,2,4,5,2,3,3,5,1,2,1,32,37.120794,2
170317702021,2,2,3,1,2,3,4,5,2,2,1,27,20.242692,2
170317702022,2,2,3,2,2,3,4,5,2,2,1,28,23.607281,2


### 3. process demographic columns

pop_indicators = [
    'LOWINCPCT',
    'MINORPCT',
    'LESSHSPCT',
    'LINGISOPCT',
    'UNDER5PCT',
    'OVER64PCT'
]

pop_dict = {
    'LOWINCPCT': 'LWINCPCT',
    'MINORPCT': 'MINORPCT',
    'LESSHSPCT': 'LESHSPCT',
    'LINGISOPCT': 'LNGISPCT',
    'UNDER5PCT': 'UNDR5PCT',
    'OVER64PCT': 'OVR64PCT'
}

display(df_dict[df_dict.FIELD_NAME.isin(pop_indicators)])

#### 3a. calculate percentiles


c_pop_df = df.filter(pop_indicators).apply(lambda series: series.apply(lambda x: stats.percentileofscore(series, x, kind='strict')))
c_pop_df.rename(lambda x: 'C_' + pop_dict.get(x,x), axis='columns', inplace=True)

display(c_pop_df.head())

#### 3b. bin them into quintiles

bins = [-1,20,40,60,80,100]
labels = [1,2,3,4,5]

s_pop_df = c_pop_df.apply(lambda series: pd.cut(series, bins, labels=labels).values.add_categories(0).astype(int))
s_pop_df.rename(lambda x: x.replace('C_', 'S_'), axis='columns', inplace=True)

display(s_pop_df.head())

#### 3c. sum up quintiles and bin them too

s_pop_df['S_POP_TOTAL'] = s_pop_df.sum(axis=1)

series = s_pop_df['S_POP_TOTAL']
s_pop_df['C_POP_TOTAL'] = series.apply(lambda x: stats.percentileofscore(series, x, kind='strict'))
s_pop_df['S_POP_OVERALL'] = pd.cut(s_pop_df['C_POP_TOTAL'], bins, labels=labels).astype(int)

display(s_pop_df.head())

### 4. merge everything

merged_df = pd.concat([df, c_pop_df, c_env_df, s_pop_df, s_env_df], axis=1, sort=True)
merged_df['S_OVERALL'] = merged_df['S_POP_OVERALL'] + merged_df['S_ENV_OVERALL']

display(merged_df.head())

### 5. export

merged_df.to_file('ej_suburban_v3/output_ej.shp')

In [18]:
merged_df = pd.concat([df, s_env_df], axis=1, sort=True)
merged_df.to_file('0524 map/env/suburban/suburban.shp')